# Spaceship Titanic
[Kaggle Competiton](https://www.kaggle.com/competitions/spaceship-titanic/)


    - PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
    - HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
    - CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
    - Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
    - Destination - The planet the passenger will be debarking to.
    - Age - The age of the passenger.
    - VIP - Whether the passenger has paid for special VIP service during the voyage.
    - RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
    - Name - The first and last names of the passenger.
    - Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.



In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score


def convert(dataframe):
    dataframe["HomePlanet"]=dataframe["HomePlanet"].astype("category")
    dataframe["Destination"]=dataframe["Destination"].astype("category")
    dataframe[["group", "number_within_group"]]=dataframe["PassengerId"].str.split("_", expand=True)
    dataframe["group"]=dataframe["group"].astype(int)
    dataframe["number_within_group"]=dataframe["number_within_group"].astype(int)
    dataframe[["deck", "num", "side"]]=dataframe["Cabin"].str.split("/", expand=True)
    dataframe["deck"]=dataframe["deck"].astype("category")
    dataframe["num"]=dataframe["num"].astype("category")
    dataframe["side"]=dataframe["side"].astype("category")
    dataframe = dataframe.drop(["Cabin", "PassengerId", "Name"], axis=1) #also remove name
    dataframe["CryoSleep"] =dataframe["CryoSleep"].astype(bool)
    dataframe["VIP"] =dataframe["VIP"].astype(bool)
    dataframe["Age"]=pd.to_numeric(dataframe['Age'], errors='coerce', downcast='integer')
    dataframe["RoomService"]=pd.to_numeric(dataframe['RoomService'], errors='coerce', downcast='integer')
    dataframe["FoodCourt"]=pd.to_numeric(dataframe['FoodCourt'], errors='coerce', downcast='integer')
    dataframe["ShoppingMall"]=pd.to_numeric(dataframe['ShoppingMall'], errors='coerce', downcast='integer')
    dataframe["Spa"]=pd.to_numeric(dataframe['Spa'], errors='coerce', downcast='integer')
    dataframe["VRDeck"]=pd.to_numeric(dataframe['VRDeck'], errors='coerce', downcast='integer')
    return dataframe


mnist_train = pd.read_csv("/home/rainer/Downloads/ML_datasets/spaceship_titanic_train.csv")
mnist_train = convert(mnist_train)
mnist_train["Transported"] =mnist_train["Transported"].astype(bool)

X, Y = mnist_train.drop('Transported', axis=1), mnist_train[['Transported']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
dtrain_reg = xgb.DMatrix(X_train, Y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, Y_test, enable_categorical=True)

In [2]:
def objective(trial):
    param = {
        'tree_method':'gpu_hist',
        "objective": "binary:hinge",
        'eta': trial.suggest_float('eta', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.01, 10, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1),

    }

    model = xgb.train(
        params=param,
        dtrain=dtrain_reg,
        num_boost_round=200,
    )
    preds = model.predict(dtest_reg)
    accuracy = accuracy_score(Y_test, preds)
    return accuracy

In [3]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2000)
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-04-28 22:50:32,415] A new study created in memory with name: no-name-046714f8-2902-4ddd-8541-303e8ec63acf
[I 2023-04-28 22:50:33,203] Trial 0 finished with value: 0.765869365225391 and parameters: {'eta': 0.28339515915542934, 'max_depth': 3, 'min_child_weight': 0.20217514821454285, 'subsample': 0.8180419218327462, 'colsample_bynode': 0.18777554772009136}. Best is trial 0 with value: 0.765869365225391.
[I 2023-04-28 22:50:33,834] Trial 1 finished with value: 0.797608095676173 and parameters: {'eta': 0.04526594601622943, 'max_depth': 3, 'min_child_weight': 1.9922194228955081, 'subsample': 0.13807759905190864, 'colsample_bynode': 0.2598109127410959}. Best is trial 1 with value: 0.797608095676173.
[I 2023-04-28 22:50:34,643] Trial 2 finished with value: 0.7359705611775529 and parameters: {'eta': 0.44020842888217065, 'max_depth': 6, 'min_child_weight': 0.0683337411561664, 'subsample': 0.9869077401593852, 'colsample_bynode': 0.49186631733595587}. Best is trial 1 with value: 0.7976080

Number of finished trials: 2000
Best trial:
  Value: 0.8201471941122355
  Params: 
    eta: 0.15190769404863355
    max_depth: 8
    min_child_weight: 9.727738443777076
    subsample: 0.3181317798482385
    colsample_bynode: 0.10230354506932182
